In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib_inline.backend_inline import set_matplotlib_formats
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import calinski_harabasz_score
from typing import Dict, Optional, Tuple, Any, List
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
from sklearn.metrics import precision_score
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.inspection import permutation_importance
from scipy import stats
from collections import defaultdict
from math import log2
from scipy.sparse import coo_matrix
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.stats import spearmanr
import re, gc, hashlib
from typing import Optional, Iterable
from time import perf_counter
from datetime import datetime, timedelta
from sklearn.metrics import ndcg_score
import warnings, math, json, os
from sklearn.linear_model import LogisticRegression
from typing import *
from xgboost import XGBRanker, XGBRegressor
from catboost import CatBoostRanker, CatBoostRegressor, Pool

# def _rmse(y_true, y_pred):
#     return float(np.sqrt(mean_squared_error(y_true, y_pred)))

# def _spearman(y_true, y_pred):
#     c = spearmanr(y_true, y_pred).correlation
#     return float(c) if np.isfinite(c) else np.nan

# import warnings
# warnings.filterwarnings('ignore')


In [2]:
ads_sch = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/스케줄.csv')
print(f"데이터 수 : {ads_sch.shape}")
ads_sch.head()

데이터 수 : (527696, 11)


,sch_idx,ads_idx,sch_type,mda_idx_arr,sch_sdatetime,sch_edatetime,sch_adv_pay,sch_ads_pay,sch_mda_pay,sch_clk_num,sch_turn_num
0,1257478,463989,M,"562,563",2025-09-11 09:42:03,9999-12-31 23:59:59,18,18,14,99999999,14
1,1257477,463989,A,NaN,2025-09-11 09:42:03,9999-12-31 23:59:59,18,18,12,99999999,14
2,1257476,463988,M,"562,563",2025-09-11 09:42:02,9999-12-31 23:59:59,18,18,14,99999999,18
3,1257475,463988,A,NaN,2025-09-11 09:42:02,9999-12-31 23:59:59,18,18,12,99999999,18
4,1257474,463987,M,"562,563",2025-09-11 09:41:55,9999-12-31 23:59:59,18,18,14,99999999,7


In [3]:
ads_time = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정_시간별적립보고서(최종).csv')
print(f"데이터 수 : {ads_time.shape}")
ads_time.head()

데이터 수 : (279916, 10)


,rpt_time_date,rpt_time_time,ads_idx,mda_idx,rpt_time_clk,rpt_time_turn,rpt_time_acost,rpt_time_earn,weekday,week_type
0,2025-07-27,0,11059,398,5,0,0,0,6,주말
1,2025-07-27,0,13209,583,3,0,0,0,6,주말
2,2025-07-27,0,13928,303,3,3,510,360,6,주말
3,2025-07-27,0,13928,324,3,3,510,360,6,주말
4,2025-07-27,0,13928,340,6,6,1020,720,6,주말


In [4]:
ads_domain = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/광고도메인리스트_1년치.csv')
print(f"데이터 수 : {ads_domain.shape}")
ads_domain.head()

데이터 수 : (233408, 24)


,Unnamed: 0,domain,row_id,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,...,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate
0,0,금융,153088,154946,KoM69LPFUB,8,50,3,7,어센틱금융그룹 내보험조회,...,2025-02-12 10:00:00,2029-11-29 00:00:00,0,100,7,800,640,2444400,NONE,2025-02-12 10:18:46
1,21,게임,135518,137375,RBjgpLVOmR,71,910,3,8,DK모바일 리본 사전예약,...,2025-01-10 18:00:00,2030-01-01 00:00:00,0,100,7,200,150,2431500,NONE,2025-01-10 18:48:52
2,23,게임,427141,429013,TUSVjcBYtK,76,1355,3,5,인생존망겜(레벨 67 달성),...,2025-07-11 16:23:32,9999-12-31 23:59:59,0,100,2,9600,7200,2460500,NONE,2025-07-11 16:23:32
3,31,게임,161321,163182,OLNpNxxGJm,76,1355,3,5,고블린 우드: 타이쿤 유휴 심,...,2025-02-27 16:00:00,9999-12-31 23:59:59,0,100,2,4480,3360,2451800,NONE,2025-02-26 17:05:49
4,34,게임,161392,163254,r78MSmvuQk,76,1355,3,5,Save the Pets,...,2025-02-27 16:00:00,2031-01-03 00:00:00,0,100,1,1440,1080,2450800,NONE,2025-02-26 17:57:33


In [5]:
ads_list = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정2_광고목록.csv')
print(f"데이터 수 : {ads_list.shape}")
ads_list.head()

데이터 수 : (445039, 21)


,ads_idx,ads_code,aff_idx,adv_idx,ads_type,ads_category,ads_name,ads_icon_img,ads_summary,ads_save_way,...,ads_sdate,ads_edate,ads_age_min,ads_age_max,ads_os_type,ads_contract_price,ads_reward_price,ads_order,ads_rejoin_type,regdate
0,160,tr71opm6C1,6,10,1,1,리니지레드나이츠,https://lh3.googleusercontent.com/oDGXxngO9oOP...,[앱설명]<br />[설치형] 매력만점 영웅들과 소환수들이 함께하는 취향저격 RPG...,받기,...,2016-12-01 00:00:00,9999-12-31 23:59:59,0,100,2,170,150,982800,NONE,2017-04-06 13:19:53
1,284,xlNpfkgtLX,6,10,1,1,강철의함대:Ocean Overlord,http://nextapps-nas.aws.appang.kr/icon/2016122...,"[앱설명]<br />[설치형] 한, 중, 일 아시아 유저들과 함께 대규모 해상 전투...",받기,...,2016-12-29 00:00:00,9999-12-31 23:59:59,0,100,2,180,160,987200,NONE,2017-01-10 10:37:22
2,292,7Hj8V5QKiy,6,10,1,1,스노우 SNOW,http://nextapps-nas.aws.appang.kr/icon/2016092...,"[앱설명]<br />[설치형] 셀카, 얼굴인식 스티커, 꿀잼 카메라<br /><br...",받기,...,2016-12-29 00:00:00,9999-12-31 23:59:59,0,100,2,160,160,987300,NONE,2017-01-02 12:51:34
3,304,1RfxHJcuEe,6,10,2,1,서머너즈 워: 천공의 아레나,http://nextapps-nas.aws.appang.kr/icon/EecpHzP...,[앱설명]<br />[실행형] 전 세계 5000만 소환사들을 사로잡은 명작 RPG<...,최초 오픈,...,2017-01-01 00:00:00,9999-12-31 23:59:59,0,100,2,230,210,931000,NONE,2017-03-30 17:05:23
4,306,l3GyilYEVk,6,10,1,1,하이마트,http://nextapps-nas.aws.appang.kr/icon/2016032...,새롭게 단장한 하이마트 쇼핑몰 앱!,받기,...,2017-01-01 00:00:00,2019-11-01 00:00:00,0,100,2,170,140,1155100,NONE,2017-06-12 15:24:46


In [6]:
ads_rwd = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정_광고적립.csv')
print(f"데이터 수 : {ads_rwd.shape}")
ads_rwd.head()

데이터 수 : (1475031, 14)


,rwd_idx,ads_idx,ads_code,mda_idx,pub_sub_rel_id,dvc_idx,advid,click_key,adv_cost,earn_cost,click_date,regdate,ctit,cost_diff
0,238304818,413490,CFZ6lzEmeD,22,27496512,32947806,09b6a808-4f76-4584-87c5-626f21f472a0,b0e22dba943810794d25c993dff425c4e16d0744,190,120,2025-07-26 00:00:31,2025-07-26 00:05:02,271,70
1,238304829,413490,CFZ6lzEmeD,22,27496512,47387244,ba4fd0c8-4f06-4dfa-9c3f-c6b39ee7066e,0679ff4c6a14beba296e44d7ffdd88129f42ffbf,190,120,2025-07-26 00:00:32,2025-07-26 00:08:17,465,70
2,238304816,413490,CFZ6lzEmeD,22,27496512,32425124,64c675d0-8edd-4478-9906-a806651fdf92,68eb28d636724061da21e490a1afff6a39cd78f7,190,120,2025-07-26 00:00:42,2025-07-26 00:04:43,241,70
3,238304817,413490,CFZ6lzEmeD,22,27496513,34006545,a08ebb04-e70a-4492-89d4-17fa96bf8c75,37da42f12b6f732d8c8bf1a609c01e569b7f8a09,190,120,2025-07-26 00:00:54,2025-07-26 00:04:45,231,70
4,238304802,133653,rl8ud4uNQC,801,1,56867331,dbe54f40-41b5-4423-ae2f-048fd6ecf5af,624b1bbc57c9ea81005c74e1956d48e76d974816,260,195,2025-07-26 00:01:07,2025-07-26 00:02:04,57,65


In [7]:
ads_info = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/수정2_광고참여정보.csv')
print(f"데이터 수 : {ads_info.shape}")
ads_info.head()

데이터 수 : (16854865, 14)


,click_key,ads_idx,dvc_idx,mda_idx,pub_sub_rel_id,contract_price,media_price,click_day,click_time,click_date,exp_day,network,user_ip,converted
0,000000d54b9faad47ee99d6cd3cf53894dd4baa5,313780,61906528,539,144350110,6000,4500,2025-08-17,21,2025-08-17 21:07:37,2025-09-16,0,16.184.28.219,0
1,000002b4d92f7648b455877c2676452efcd22a09,412426,34422806,58,46032732,180,170,2025-07-26,2,2025-07-26 02:18:24,2025-08-25,NaN,35.78.117.76,1
2,0000057e97361ff3d0263aaecee34cfaa3ba30fb,443660,38366075,808,1,170,120,2025-08-12,18,2025-08-12 18:17:59,2025-09-11,WIFI,39.7.55.192,1
3,00000607f60139015da3ee1dd5499db3faa100dc,360192,61894110,539,144350110,6000,4500,2025-08-17,3,2025-08-17 03:35:02,2025-09-16,0,211.252.100.157,0
4,0000066bc25d4a6d147c27326cf972a4de88024e,372307,61956954,539,144350110,15600,11700,2025-08-18,8,2025-08-18 08:01:26,2025-09-17,0,3.38.148.211,0


In [8]:
ads_join = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/참여적립조인.csv')
print(f"데이터 수 : {ads_join.shape}")
ads_join.head()

데이터 수 : (16854865, 22)


,Unnamed: 0,click_key,ads_idx,dvc_idx,mda_idx,pub_sub_rel_id,contract_price,media_price,click_day,click_time,...,network,user_ip,rwd_idx,ads_code,advid,adv_cost,earn_cost,regdate,ctit,conversion
0,0,000000d54b9faad47ee99d6cd3cf53894dd4baa5,313780,61906528,539,144350110,6000,4500,2025-08-17,21,...,0,16.184.28.219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,000002b4d92f7648b455877c2676452efcd22a09,412426,34422806,58,46032732,180,170,2025-07-26,2,...,NaN,35.78.117.76,238313226.0,yuRe4YoPuZ,40ceb01c-9b60-4452-bacd-0e061fb18ab4,180.0,170.0,2025-07-26 02:19:21,57.0,1
2,2,0000057e97361ff3d0263aaecee34cfaa3ba30fb,443660,38366075,808,1,170,120,2025-08-12,18,...,WIFI,39.7.55.192,239017177.0,WuIp8HKmt9,73793f44-a471-4351-ae1c-d09accaf223a,170.0,120.0,2025-08-12 18:19:21,82.0,1
3,3,00000607f60139015da3ee1dd5499db3faa100dc,360192,61894110,539,144350110,6000,4500,2025-08-17,3,...,0,211.252.100.157,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,4,0000066bc25d4a6d147c27326cf972a4de88024e,372307,61956954,539,144350110,15600,11700,2025-08-18,8,...,0,3.38.148.211,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [9]:
ads_segment = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/ads_segment.csv')
print(f"데이터 수 : {ads_segment.shape}")
ads_segment.head()

데이터 수 : (9517, 40)


,Unnamed: 0,ads_idx,media_count,user_count,total_clicks,total_conversions,ads_category,domain,ads_os_type,ads_order,...,performance_level,ads_level,media_score,conv_score,clicks_score,stability_score,cvr_score,total_score,ads_size,cluster
0,0,6508,1,1,2,0,10,금융,7,2381500,...,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0
1,1,6985,2,30,46,0,7,금융,7,1201938,...,LOW,MEDIUM_LOW,2,0,0,3,0,5,MEDIUM,0
2,2,8327,1,6,11,0,10,미디어/컨텐츠,7,1673458,...,LOW,SMALL_LOW,1,0,0,3,0,4,SMALL,0
3,3,9264,1,1,1,0,10,금융,7,2372800,...,LOW,SMALL_LOW,1,0,0,0,0,1,SMALL,0
4,4,9716,2,4,6,1,8,생활,2,2699900,...,LOW,MEDIUM_LOW,2,0,1,1,0,4,SMALL,0


In [10]:
media_portfolio = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/media_portfolio.csv')
print(f"데이터 수 : {media_portfolio.shape}")
media_portfolio.head()

데이터 수 : (189, 50)


,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,LARGE,...,domain_비영리/공공_pct,domain_생활_pct,domain_식당/카페_pct,domain_식음료_pct,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit
0,0,12,75700,232438,82938,2025-07-26 00:00:02,2025-08-25 11:22:16,31,2675.419355,7,...,16.666667,12.500000,0.00000,16.666667,0.000000,0.000000,4.166667,0.000000,12.500000,3003668
1,1,14,31834,84763,22983,2025-07-26 00:01:22,2025-08-25 11:22:40,31,741.387097,31,...,0.000000,13.559322,0.00000,3.389831,0.000000,0.000000,3.389831,1.694915,5.084746,2733547
2,2,18,2825,5961,250,2025-07-26 00:00:49,2025-08-25 11:13:19,31,8.064516,19,...,0.000000,8.695652,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,4.347826,182366
3,3,22,37668,83355,35232,2025-07-26 00:00:31,2025-08-25 11:14:50,31,1136.516129,67,...,3.012048,11.445783,0.60241,4.216867,1.204819,7.831325,4.819277,1.204819,7.228916,2541770
4,4,26,402,678,70,2025-07-26 01:45:31,2025-08-25 10:22:36,31,2.258065,10,...,0.000000,10.526316,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,10.526316,2690


In [11]:
ads_new = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/신규가상광고.csv')
print(f"데이터 수 : {ads_new.shape}")
ads_new.head()

데이터 수 : (8, 10)


,ads_idx,ads_type,ads_category,ads_name,ads_os_type,ads_contract_price,ads_media_price,ads_reward_price,ads_rejoin_type,domain
0,500000,12,11,청정원 선물세트,7,6181,4000,4000,REJOINABLE,식음료
1,500001,3,4,옹짬뽕 포항양덕본점,7,18,14,12,ADS_CODE_DAILY_UPDATE,식당/카페
2,500002,3,4,추석선물세트 (리뷰 전체보기 클릭 후 URL 제출),7,17,8,8,NONE,식당/카페
3,500003,11,3,네이버 뉴스 제주뉴스 구독,7,230,160,180,NONE,미디어/컨텐츠
4,500004,7,3,폴라베어 인스타그램 구독,1,230,160,180,NONE,커머스


In [12]:
media_performance_classification = pd.read_csv('/Users/sparta/Documents/GitHub/final_project/data/media_performance_classification.csv')
print(f"데이터 수 : {media_performance_classification.shape}")
media_performance_classification.head(20)

데이터 수 : (188, 55)


,Unnamed: 0.1,Unnamed: 0,mda_idx,user_count,total_clicks,total_conversions,first_click,last_click,days_active,daily_avg_conversions,...,domain_운동/스포츠_pct,domain_운세_pct,domain_의료/건강_pct,domain_채용_pct,domain_커머스_pct,expected_total_profit,conversion_rate,last_click_dt,classification,basic_classification
0,0,0,12,75700,232438,82938,2025-07-26 00:00:02,2025-08-25 11:22:16,31,2675.419355,...,0.000000,0.000000,4.166667,0.000000,12.500000,3003668,0.356818,2025-08-25 11:22:16,특화전문형_MEGA특화,특화전문형
1,1,1,14,31834,84763,22983,2025-07-26 00:01:22,2025-08-25 11:22:40,31,741.387097,...,0.000000,0.000000,3.389831,1.694915,5.084746,2733547,0.271144,2025-08-25 11:22:40,안정공급형,안정공급형
2,2,2,18,2825,5961,250,2025-07-26 00:00:49,2025-08-25 11:13:19,31,8.064516,...,0.000000,0.000000,0.000000,0.000000,4.347826,182366,0.041939,2025-08-25 11:13:19,특화전문형_게임특화,특화전문형
3,3,3,22,37668,83355,35232,2025-07-26 00:00:31,2025-08-25 11:14:50,31,1136.516129,...,1.204819,7.831325,4.819277,1.204819,7.228916,2541770,0.422674,2025-08-25 11:14:50,대량처리형,대량처리형
4,4,4,26,402,678,70,2025-07-26 01:45:31,2025-08-25 10:22:36,31,2.258065,...,0.000000,0.000000,0.000000,0.000000,10.526316,2690,0.103245,2025-08-25 10:22:36,특화전문형_게임특화,특화전문형
5,5,5,30,3606,5160,391,2025-07-26 00:42:21,2025-08-25 11:06:44,31,12.612903,...,0.000000,3.333333,1.666667,3.333333,3.333333,22520,0.075775,2025-08-25 11:06:44,안정공급형(후보),안정공급형(후보)
6,6,6,32,870,1574,163,2025-07-26 10:48:49,2025-08-25 10:38:39,30,5.433333,...,0.000000,0.000000,14.285714,0.000000,14.285714,191080,0.103558,2025-08-25 10:38:39,관리 필요,관리 필요
7,7,7,54,6395,12838,4299,2025-07-26 00:12:15,2025-08-25 11:11:45,31,138.677419,...,1.510574,4.229607,9.667674,0.906344,24.773414,309633,0.334865,2025-08-25 11:11:45,안정공급형(후보),안정공급형(후보)
8,8,8,56,17274,40127,9484,2025-07-26 00:04:02,2025-08-25 11:18:47,31,305.935484,...,0.000000,11.111111,5.555556,0.000000,5.555556,580630,0.236350,2025-08-25 11:18:47,안정공급형(후보),안정공급형(후보)
9,9,9,58,169122,482794,155827,2025-07-26 00:01:29,2025-08-25 11:24:04,31,5026.677419,...,0.000000,7.692308,3.846154,0.000000,3.846154,3884970,0.322761,2025-08-25 11:24:04,안정공급형(후보),안정공급형(후보)


In [13]:
ㅋㅋ

NameError: name 'ᄏᄏ' is not defined

In [ ]:
# -*- coding: utf-8 -*-
# =========================================================
#  Ad × Media 추천 + 전환수 예측 (XGBoostRanker + XGBRegressor)
#  - 누수 방지: 모든 피처는 OBS에서만 생성, 라벨은 TGT
#  - 클릭: Tweedie 회귀, CVR: logit 회귀
#  - 예측: yhat_turn = yhat_click × yhat_cvr
#  - 계약종료형(banned_mda) 제거, 현재 진행중 페어(running_pairs)는 포함
# =========================================================
# 필요 패키지:
#   pip install pandas numpy pyarrow scikit-learn xgboost
# =========================================================

warnings.filterwarnings("ignore")

# -------------------------------
# 유틸
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")

def _norm_cat_col(df: pd.DataFrame, col: str) -> pd.DataFrame:
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
    return df

def _norm_cat(s): return pd.Series(s, copy=False).astype("string").str.strip().str.lower()

def _pick_col(cols, cand):
    for c in cand:
        if c in cols: return c
    return None

def _ensure_col(df: pd.DataFrame, out_col: str, cands: List[str]) -> pd.DataFrame:
    col = _pick_col(df.columns, cands)
    df = df.copy()
    if col is None: df[out_col] = "__na__"
    else:           df[out_col] = _norm_cat(df[col])
    return df

def _dedup_running_pairs(running_pairs: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
    if running_pairs is None or running_pairs.empty:
        return None
    rp = running_pairs.copy()
    rp["ads_idx"] = rp["ads_idx"].astype(int)
    rp["mda_idx"] = rp["mda_idx"].astype(int)
    rp = rp.drop_duplicates(["ads_idx","mda_idx"])
    return rp

# -------------------------------
# 1) 윈도우
# -------------------------------
def split_obs_tgt_by_dates(
    ads_time: pd.DataFrame,
    obs_start: str = "2025-07-25",
    tgt_start: str = "2025-08-20",s
    obs_end: Optional[str] = None,
    tgt_end: Optional[str] = None,
    tgt_max_days: int = 6
):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    if t["date"].isna().all():
        raise ValueError("ads_time['rpt_time_date']에 유효한 날짜가 없습니다.")
    obs_start = pd.to_datetime(obs_start).normalize()
    tgt_start = pd.to_datetime(tgt_start).normalize()
    obs_end = (pd.to_datetime(obs_end).normalize() if obs_end is not None
               else (tgt_start - pd.Timedelta(days=1)).normalize())
    last_date = t["date"].max().normalize()
    tgt_end = (pd.to_datetime(tgt_end).normalize() if tgt_end is not None
               else min(last_date, (tgt_start + pd.Timedelta(days=tgt_max_days)).normalize()))
    obs = t[(t["date"] >= obs_start) & (t["date"] <= obs_end)].copy()
    tgt = t[(t["date"] >= tgt_start) & (t["date"] <= tgt_end)].copy()
    return obs, tgt, obs_start, tgt_end, (tgt_start - pd.Timedelta(days=1)).normalize()

# -------------------------------
# 2) 시간/매체/카테고리/도메인 컨텍스트 (OBS만)
# -------------------------------
def _build_time_signals(obs: pd.DataFrame) -> pd.DataFrame:
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["weekday"] = _to_dt(t["rpt_time_date"]).dt.weekday
    wd = t.groupby(["mda_idx","weekday"], as_index=False).agg(clk=("clk","sum"))
    wd["is_weekend"] = wd["weekday"].isin([5,6]).astype(int)
    g = wd.groupby("mda_idx", as_index=False).agg(clk_total=("clk","sum"))
    w = wd.merge(g, on="mda_idx", how="left")
    day_ratio = (w.assign(ratio=w["clk"]/w["clk_total"].replace(0,np.nan))
                   .sort_values(["mda_idx","ratio"], ascending=[True,False])
                   .groupby("mda_idx").head(1)[["mda_idx","ratio"]]
                   .rename(columns={"ratio":"mda_day_ratio"}))
    weekend_ratio = (w.groupby(["mda_idx","is_weekend"], as_index=False)
                       .agg(clk=("clk","sum")).merge(g,on="mda_idx",how="left"))
    weekend_ratio["mda_weekend_ratio"] = (weekend_ratio["clk"] /
                                          weekend_ratio["clk_total"].replace(0,np.nan)).fillna(0)
    weekend_ratio = weekend_ratio[weekend_ratio["is_weekend"]==1][["mda_idx","mda_weekend_ratio"]]
    out = day_ratio.merge(weekend_ratio, on="mda_idx", how="outer").fillna(0.0)
    out["mda_idx"] = out["mda_idx"].astype(int)
    return out

def build_attr_context(obs: pd.DataFrame, ads_list_with_domain: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))

    # mda 전역
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"), mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0,np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std(ddof=0)
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    # 카테고리×매체
    al = ads_list_with_domain[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])
    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"]/cat_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot = cat_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    cat_mda = cat_mda.merge(mda_tot, on="mda_idx", how="left")
    cat_mda["cat_share"] = (cat_mda["conv"]/cat_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 도메인×매체
    t_dom = t.merge(
        ads_list_with_domain[["ads_idx","ads_domain"]].drop_duplicates("ads_idx"),
        on="ads_idx", how="left"
    )
    t_dom["ads_domain"] = _norm_cat(t_dom["ads_domain"])
    dom_mda = (t_dom.groupby(["mda_idx","ads_domain"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    dom_mda["dom_cvr"] = (dom_mda["conv"]/dom_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot2 = dom_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    dom_mda = dom_mda.merge(mda_tot2, on="mda_idx", how="left")
    dom_mda["dom_share"] = (dom_mda["conv"]/dom_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    # 시간 신호
    time_sig = _build_time_signals(obs)

    # (선택) 광고주 신호
    adv_ctx = None
    if "adv_idx" in ads_list_with_domain.columns:
        t_adv = t.merge(ads_list_with_domain[["ads_idx","adv_idx"]].drop_duplicates("ads_idx"),
                        on="ads_idx", how="left")
        adv_all = (t_adv.groupby("adv_idx", as_index=False)
                        .agg(adv_clk_global=("clk","sum"),
                             adv_conv_global=("conv","sum")))
        adv_all["adv_cvr_global"] = (adv_all["adv_conv_global"] /
                                     adv_all["adv_clk_global"].replace(0,np.nan)).fillna(0.0)
        mu2, sd2 = adv_all["adv_conv_global"].mean(), adv_all["adv_conv_global"].std(ddof=0)
        adv_all["adv_pop_norm"] = (adv_all["adv_conv_global"] - mu2) / (sd2 + 1e-9)
        adv_ctx = adv_all

    return {"mda_all": mda_all, "cat_mda": cat_mda, "dom_mda": dom_mda,
            "time_sig": time_sig, "adv_ctx": adv_ctx}

# -------------------------------
# 3) 학습 대상 광고
# -------------------------------
def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = 20000, rng_seed: int = 42) -> np.ndarray:
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1,int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

# -------------------------------
# 4) 후보 생성
# -------------------------------
def create_candidates(ads_list: pd.DataFrame, mda_all: pd.DataFrame, cat_mda: pd.DataFrame,
                      topC=10, add_random=2, cat_topM=5,
                      min_active_clk=200, min_active_conv=20,
                      rng_seed=42, ad_ids_filter=None, max_per_ad=20):
    mda_active = mda_all[(mda_all["mda_clk_global"]>=min_active_clk) |
                         (mda_all["mda_conv_global"]>=min_active_conv)].copy()
    if mda_active.empty: mda_active = mda_all.copy()
    mda_ids_global = (mda_active.sort_values("mda_conv_global", ascending=False)["mda_idx"]
                      .astype(int).values)
    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True,False])
                      .groupby("ads_category").head(cat_topM)[["ads_category","mda_idx"]])
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c,g in cat_top.groupby("ads_category")}
    rng = np.random.RandomState(rng_seed)
    ad_ids = (np.array(ad_ids_filter, dtype=int) if (ad_ids_filter is not None and len(ad_ids_filter)>0)
              else ads_list["ads_idx"].dropna().astype(int).unique())
    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_idx"] = ad_meta["ads_idx"].astype(int)
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs=[]
    for ad in ad_ids:
        base=[]
        cat = ad2cat.get(int(ad), None)
        if cat in cat2top: base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC,len(mda_ids_global))])
        base=list(dict.fromkeys(base))
        remain=np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain)>0 and add_random>0:
            base.extend(rng.choice(remain, size=min(add_random,len(remain)), replace=False).tolist())
        if len(base)>max_per_ad: base=base[:max_per_ad]
        pairs += [(int(ad), int(m)) for m in base]
    out = pd.DataFrame(pairs, columns=["ads_idx","mda_idx"]).drop_duplicates()
    print(f"[Candidates] pairs={len(out):,} | ads={len(ad_ids):,} | active_mda={len(mda_active):,} | topC={topC}, rnd={add_random}, cat_topM={cat_topM}, per_ad≤{max_per_ad}")
    return out

# -------------------------------
# 5) 피처 & 라벨
# -------------------------------
FEAT_ALLOW = [
    "mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm",
    "cat_cvr","cat_share",
    "price_reward_log","price_contract_log","price_ratio",
    "dom_cvr","dom_share",
    "mda_day_ratio","mda_weekend_ratio",
    "adv_cvr_global","adv_pop_norm",
    "mda_size","active_ratio","day_ratio","weekend_ratio"
]

def prepare_training_attr_only(
    obs: pd.DataFrame, tgt: pd.DataFrame, ads_list_with_domain: pd.DataFrame,
    topC=10, add_random=2, cat_topM=5, min_active_clk=200, min_active_conv=20,
    max_train_ads: Optional[int] = 20000, min_pos_in_tgt: int = 1,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None,
    running_pairs: Optional[pd.DataFrame] = None
):
    running_pairs = _dedup_running_pairs(running_pairs)

    if excluded_mda_ids:
        obs = obs[~obs["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()
        tgt = tgt[~tgt["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    train_ads = pick_train_ads_from_tgt(tgt, min_pos_in_tgt=min_pos_in_tgt, max_train_ads=max_train_ads)
    actx = build_attr_context(obs, ads_list_with_domain)
    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]

    cand = create_candidates(
        ads_list=ads_list_with_domain, mda_all=mda_all, cat_mda=cat_mda,
        topC=topC, add_random=add_random, cat_topM=cat_topM,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv,
        ad_ids_filter=train_ads, max_per_ad=20
    )

    # 현재 진행중 페어는 포함(제외하지 않음)
    if excluded_mda_ids:
        cand = cand[~cand["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    df = cand.merge(
        mda_all[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]],
        on="mda_idx", how="left"
    )

    ad_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    ad_meta = _norm_cat_col(ad_meta, "ads_category")
    ad_meta = _ensure_col(ad_meta, "ads_domain", ["ads_domain","domain"])
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = _to_num(ad_meta[c])
    ad_meta["price_reward_log"]   = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_contract_log"] = np.log1p(ad_meta["ads_contract_price"])
    ad_meta["price_ratio"]        = (ad_meta["ads_reward_price"] /
                                     ad_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)

    df = df.merge(ad_meta[["ads_idx","ads_category","ads_domain","adv_idx",
                           "price_reward_log","price_contract_log","price_ratio"]],
                  on="ads_idx", how="left")

    df      = _norm_cat_col(df, "ads_category")
    cat_mda = _norm_cat_col(cat_mda, "ads_category")
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]],
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0,"cat_share":0.0})

    dom_mda = dom_mda.copy()
    dom_mda["ads_domain"] = _norm_cat(dom_mda["ads_domain"])
    df["ads_domain"] = _norm_cat(df["ads_domain"])
    df = df.merge(dom_mda[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                  on=["mda_idx","ads_domain"], how="left").fillna({"dom_cvr":0.0,"dom_share":0.0})

    time_sig = actx["time_sig"]
    df = df.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if actx["adv_ctx"] is not None:
        adv_ctx = actx["adv_ctx"].copy()
        df = df.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in df.columns: df[c]=0.0
    df[["adv_cvr_global","adv_pop_norm"]] = df[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: df = df.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in df.columns: df[c]=0.0

    # ---- 라벨(TGT)
    tgt2 = tgt.copy()
    tgt2["clk"]  = _to_num(tgt2.get("rpt_time_clk", 0))
    tgt2["turn"] = _to_num(tgt2.get("rpt_time_turn",0))
    y_agg = (tgt2.groupby(["ads_idx","mda_idx"], as_index=False)
                   .agg(tgt_clk=("clk","sum"), tgt_turn=("turn","sum")))
    y_agg["tgt_cvr"] = (y_agg["tgt_turn"] / y_agg["tgt_clk"].replace(0,np.nan)).clip(0,1)

    df = df.merge(y_agg, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_clk":0.0,"tgt_turn":0.0})

    # 랭커용 relevance
    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum()>0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int)+1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df, actx

# -------------------------------
# 6) 학습기(XGBoost)
# -------------------------------
LABEL_GAIN = [0.0,1.0,4.0,12.0]

def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c]=0.0
        if not np.issubdtype(X[c].dtype, np.number):
            X[c]=pd.to_numeric(X[c], errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

def downsample_negatives_per_ad(df, ratio=4, random_state=42):
    out, rng = [], np.random.RandomState(random_state)
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"]>0]; neg = g[g["tgt_turn"]<=0]
        if len(pos)==0: continue
        cap = len(pos)*ratio
        if len(neg)>cap: neg = neg.sample(n=cap, random_state=rng)
        out.append(pd.concat([pos,neg], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

def train_attr_ranker_xgb(train_df: pd.DataFrame):
    feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
    X = ensure_feature_matrix(train_df, feat_cols).values
    y = train_df["relevance"].astype(int).values
    qid = train_df["ads_idx"].astype(int).values

    ranker = XGBRanker(
        objective="rank:ndcg",
        learning_rate=0.05,
        n_estimators=1200,
        max_depth=6,
        subsample=0.85,
        colsample_bytree=0.85,
        reg_lambda=1.5,
        tree_method="hist",
        random_state=42,
        verbosity=0
    )
    ranker.fit(X, y, qid=qid)
    return ranker, feat_cols

def train_click_regressor_xgb(train_df: pd.DataFrame, feat_cols: List[str], tweedie_p: float = 1.1):
    X = ensure_feature_matrix(train_df, feat_cols).values
    y = train_df["tgt_clk"].astype(float).values
    reg = XGBRegressor(
        objective="reg:tweedie",
        tweedie_variance_power=tweedie_p,
        n_estimators=1200, learning_rate=0.05, max_depth=6,
        subsample=0.85, colsample_bytree=0.85, reg_lambda=1.5,
        tree_method="hist", random_state=42, verbosity=0
    )
    reg.fit(X, y)
    return reg, {"mode":"tweedie_click","twp":tweedie_p}

def _logit_clip(p, eps=1e-6):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))
def _sigmoid(x): return 1/(1+np.exp(-x))

def train_cvr_regressor_xgb(train_df: pd.DataFrame, feat_cols: List[str]):
    g = train_df[train_df["tgt_clk"]>0].copy()
    if g.empty:
        dummy = XGBRegressor(n_estimators=10).fit(np.zeros((1, len(feat_cols))), [0.0])
        return dummy, {"mode":"cvr_logit","fallback":True, "mean_logit":-20.0}
    X = ensure_feature_matrix(g, feat_cols).values
    y = (g["tgt_turn"]/g["tgt_clk"]).clip(0,1).values
    y_logit = _logit_clip(y)

    reg = XGBRegressor(
        objective="reg:squarederror",
        n_estimators=1200, learning_rate=0.05, max_depth=6,
        subsample=0.85, colsample_bytree=0.85, reg_lambda=1.5,
        tree_method="hist", random_state=42, verbosity=0
    )
    reg.fit(X, y_logit)
    return reg, {"mode":"cvr_logit","fallback":False}

# -------------------------------
# 7) 예측
# -------------------------------
def predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_with_domain,
    K=20, batch_size=5000, max_predict_ads: Optional[int] = 20000,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None,
    running_pairs: Optional[pd.DataFrame] = None
) -> Dict[int, pd.DataFrame]:
    running_pairs = _dedup_running_pairs(running_pairs)

    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]
    time_sig, adv_ctx = actx["time_sig"], actx["adv_ctx"]

    mda_active = mda_all[(mda_all["mda_clk_global"]>=200) | (mda_all["mda_conv_global"]>=20)]
    if mda_active.empty: mda_active = mda_all
    if excluded_mda_ids:
        mda_active = mda_active[~mda_active["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    mda_active = mda_active[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]].copy()
    mda_active["mda_idx"]=mda_active["mda_idx"].astype(int)
    mda_active = mda_active.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: mda_active = mda_active.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in mda_active.columns: mda_active[c]=0.0

    ads_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    if max_predict_ads is not None: ads_meta = ads_meta.head(int(max_predict_ads))
    ads_meta = _norm_cat_col(ads_meta, "ads_category")
    ads_meta = _ensure_col(ads_meta, "ads_domain", ["ads_domain","domain"])
    ads_meta["ads_idx"] = ads_meta["ads_idx"].astype(int)
    ads_meta["ads_domain"] = _norm_cat(ads_meta["ads_domain"])
    ads_meta["ads_reward_price"]   = _to_num(ads_meta["ads_reward_price"])
    ads_meta["ads_contract_price"] = _to_num(ads_meta["ads_contract_price"])
    ads_meta["price_reward_log"]   = np.log1p(ads_meta["ads_reward_price"])
    ads_meta["price_contract_log"] = np.log1p(ads_meta["ads_contract_price"])
    ads_meta["price_ratio"]        = (ads_meta["ads_reward_price"] / ads_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)
    if adv_ctx is not None:
        ads_meta = ads_meta.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in ads_meta.columns: ads_meta[c]=0.0
    ads_meta[["adv_cvr_global","adv_pop_norm"]] = ads_meta[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    cat_mda_local = _norm_cat_col(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]].copy(), "ads_category")
    dom_mda_local = dom_mda.copy(); dom_mda_local["ads_domain"] = _norm_cat(dom_mda_local["ads_domain"])

    out={}
    ad_ids = ads_meta["ads_idx"].tolist()
    for i in range(0, len(ad_ids), batch_size):
        ad_batch = ads_meta.iloc[i:i+batch_size]
        grid = ad_batch[["ads_idx","ads_category","ads_domain","price_reward_log","price_contract_log","price_ratio",
                         "adv_idx","adv_cvr_global","adv_pop_norm"]] \
               .merge(mda_active, how="cross")

        # 진행중 페어는 포함(안티조인 없음)
        grid = grid.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0,"cat_share":0.0})
        grid = grid.merge(dom_mda_local[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                          on=["mda_idx","ads_domain"], how="left").fillna({"dom_cvr":0.0,"dom_share":0.0})

        X = ensure_feature_matrix(grid, feat_cols).values
        score = ranker.predict(X)  # 랭커 점수

        yhat_clk = np.maximum(0.0, click_reg.predict(X))
        yhat_cvr_logit = cvr_reg.predict(X)
        yhat_cvr = np.clip(_sigmoid(yhat_cvr_logit), 0.0, 1.0)
        yhat_turn = yhat_clk * yhat_cvr

        grid2 = grid[["ads_idx","mda_idx","ads_category","mda_cvr_global","mda_pop_norm","mda_clk_global","mda_conv_global"]].copy()
        grid2["score"] = score.astype(float)
        grid2["yhat_turn"] = yhat_turn.astype(float)

        for ad, g in grid2.groupby("ads_idx", sort=False):
            top = g.nlargest(K, "score")
            out[int(ad)] = top.sort_values("score", ascending=False).reset_index(drop=True)
        print(f"[Predict+Count] processed ads {i+len(ad_batch)}/{len(ad_ids)}")
    return out

# -------------------------------
# 8) 평가
# -------------------------------
def _precision_recall_hit_at_k(preds_dict, gt_dict, k=10):
    P,R,H=[],[],[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        pred = df["mda_idx"].astype(int).tolist()[:k]
        true = set(gt_dict[ad])
        if not true: continue
        inter = len(set(pred)&true)
        P.append(inter/max(1,len(pred))); R.append(inter/len(true)); H.append(1.0 if inter>0 else 0.0)
    return (float(np.mean(P)) if P else 0.0,
            float(np.mean(R)) if R else 0.0,
            float(np.mean(H)) if H else 0.0)

def _map_at_k(preds_dict, gt_dict, k=10):
    APs=[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        true=set(gt_dict[ad])
        if not true: continue
        ranked = df["mda_idx"].astype(int).tolist()[:k]
        hits=cum_prec=0
        for i,m in enumerate(ranked, start=1):
            if m in true:
                hits+=1; cum_prec += hits/i
        APs.append(cum_prec/min(len(true),k) if hits>0 else 0.0)
    return float(np.mean(APs)) if APs else 0.0

def _auc_flat(preds_dict, gt_pairs_dict, score_col="score"):
    y_true, y_score = [], []
    for ad, df in preds_dict.items():
        true_mdas = set(gt_pairs_dict.get(ad, []))
        for _, r in df.iterrows():
            y_true.append(1 if int(r["mda_idx"]) in true_mdas else 0)
            y_score.append(float(r[score_col]))
    if len(set(y_true))<2: return 0.5
    try: return float(roc_auc_score(y_true, y_score))
    except Exception: return 0.5

def evaluate_all(preds_with_counts: Dict[int, pd.DataFrame], tgt: pd.DataFrame, k=10):
    gt_pairs = tgt.groupby("ads_idx")["mda_idx"].apply(lambda s: list(pd.Series(s).dropna().astype(int).unique())).to_dict()
    prec, rec, hit = _precision_recall_hit_at_k(preds_with_counts, gt_pairs, k=k)
    mapk = _map_at_k(preds_with_counts, gt_pairs, k=k)
    auc_rank = _auc_flat(preds_with_counts, gt_pairs, score_col="score")
    auc_reg  = _auc_flat(preds_with_counts, gt_pairs, score_col="yhat_turn")
    return {f"precision@{k}":prec, f"recall@{k}":rec, f"map@{k}":mapk,
            f"hit_rate@{k}":hit, "auc_ranker":auc_rank, "auc_regressor":auc_reg}

# -------------------------------
# 9) 편의
# -------------------------------
def get_expected_conversions_for_ad(preds_with_counts: Dict[int, pd.DataFrame],
                                    ad_id: int, sort_by="yhat_turn", top=20) -> pd.DataFrame:
    df = preds_with_counts.get(int(ad_id))
    if df is None or df.empty:
        return pd.DataFrame(columns=["mda_idx","score","yhat_turn"])
    sort_by = "yhat_turn" if sort_by not in ("yhat_turn","score") else sort_by
    out = df.sort_values(sort_by, ascending=False).copy()
    return out.head(top) if top is not None else out


In [15]:
# 요구 입력 예시 변수는 동일:
#   ads_time, ads_list, ads_segment(ads_size, domain),
#   media_performance_classification(옵션), running_pairs(옵션),
#   media_portfolio(옵션)

# (A) 윈도우
obs, tgt, start, end, cut = split_obs_tgt_by_dates(ads_time, obs_start="2025-07-25", tgt_start="2025-08-20")
print(f"[Window] OBS: {start.date()} → {cut.date()}  |  TGT: {(cut+pd.Timedelta(days=1)).date()} → {end.date()}")

# (A-1) 계약종료형 매체 목록
if 'media_performance_classification' in globals():
    banned_mda = set(
        media_performance_classification
          .loc[media_performance_classification["basic_classification"].astype(str).str.strip() == "계약종료형", "mda_idx"]
          .dropna().astype(int).unique()
    )
else:
    banned_mda = set()
print(f"[Exclude] 계약종료형 mda 개수 = {len(banned_mda)}")

# (A-2) OBS/TGT에서 계약종료형 제거
if banned_mda:
    obs = obs[~obs["mda_idx"].astype(int).isin(banned_mda)].copy()
    tgt = tgt[~tgt["mda_idx"].astype(int).isin(banned_mda)].copy()

# (A-3) 현재 진행중 페어는 포함
running_pairs = _dedup_running_pairs(running_pairs) if 'running_pairs' in globals() else None
print(f"[Include] running_pairs (현재 진행중 페어) = {0 if running_pairs is None else len(running_pairs)}")

# (B) ads_list 확장
ads_list_ext = ads_list.merge(
    ads_segment[["ads_idx","ads_size","domain"]].rename(columns={"domain":"ads_domain"}),
    on="ads_idx", how="left"
)

# (C) 사이즈 필터(예: MEGA/LARGE/MEDIUM만)
size_whitelist = {"MEGA","LARGE","MEDIUM"}
ads_list_sel = ads_list_ext[
    ads_list_ext["ads_size"].astype(str).str.strip().str.upper().isin(size_whitelist)
].copy()
sel_ids = set(ads_list_sel["ads_idx"].astype(int))
tgt_sel  = tgt[tgt["ads_idx"].astype(int).isin(sel_ids)].copy()
print(f"[Filter] sizes kept = {sorted(size_whitelist)} | ads={ads_list_sel['ads_idx'].nunique():,} | tgt pairs={len(tgt_sel):,}")

# (D) 학습 데이터
train_df, actx = prepare_training_attr_only(
    obs, tgt_sel, ads_list_sel,
    topC=10, add_random=2, cat_topM=5,
    min_active_clk=200, min_active_conv=20,
    max_train_ads=20000, min_pos_in_tgt=1,
    media_portfolio=None,
    excluded_mda_ids=banned_mda,
    running_pairs=running_pairs
)
print("train_df size:", len(train_df))

# (E) 학습
train_df_ds = downsample_negatives_per_ad(train_df, ratio=4)
ranker, feat_cols   = train_attr_ranker_xgb(train_df_ds)
click_reg, click_md = train_click_regressor_xgb(train_df_ds, feat_cols, tweedie_p=1.1)
cvr_reg, cvr_md     = train_cvr_regressor_xgb(train_df_ds, feat_cols)
print("feat_cols used:", feat_cols)

# (F) 예측 & 평가
preds_with_counts = predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_sel,
    K=20, batch_size=5000, max_predict_ads=20000, media_portfolio=None,
    excluded_mda_ids=banned_mda, running_pairs=running_pairs
)
metrics_k5  = evaluate_all(preds_with_counts, tgt_sel, k=5)
metrics_k10 = evaluate_all(preds_with_counts, tgt_sel, k=10)
print("Eval@5:", metrics_k5)
print("Eval@10:", metrics_k10)

# (G) Sanity Check
leftover_tgt = tgt_sel["mda_idx"].astype(int).isin(banned_mda).sum()
print(f"[Sanity] TGT(selected sizes) 내 계약종료형 잔여: {leftover_tgt}")
any_banned_in_preds = any(
    int(m) in banned_mda
    for df in preds_with_counts.values()
    for m in df["mda_idx"].astype(int).tolist()
)
print(f"[Sanity] 예측결과에 계약종료형 포함 여부: {any_banned_in_preds}")

[Window] OBS: 2025-07-25 → 2025-08-19  |  TGT: 2025-08-20 → 2025-08-25
[Exclude] 계약종료형 mda 개수 = 51
[Include] running_pairs (현재 진행중 페어) = 0
[Filter] sizes kept = ['LARGE', 'MEDIUM', 'MEGA'] | ads=7,169 | tgt pairs=63,274
[Candidates] pairs=26,263 | ads=1,565 | active_mda=128 | topC=10, rnd=2, cat_topM=5, per_ad≤20
train_df size: 26263
feat_cols used: ['mda_clk_global', 'mda_conv_global', 'mda_cvr_global', 'mda_pop_norm', 'cat_cvr', 'cat_share', 'price_reward_log', 'price_contract_log', 'price_ratio', 'dom_cvr', 'dom_share', 'mda_day_ratio', 'mda_weekend_ratio', 'adv_cvr_global', 'adv_pop_norm', 'mda_size', 'active_ratio', 'day_ratio', 'weekend_ratio']
[Predict+Count] processed ads 5000/7169
[Predict+Count] processed ads 7169/7169
Eval@5: {'precision@5': 0.2860369609856263, 'recall@5': 0.7978426877775437, 'map@5': 0.6935171400866985, 'hit_rate@5': 0.910164271047228, 'auc_ranker': 0.8784865384628671, 'auc_regressor': 0.8664938981825647}
Eval@10: {'precision@10': 0.17027720739219715, 'reca

In [42]:

def _build_eval_frame(preds_with_counts: dict, tgt: pd.DataFrame) -> pd.DataFrame:
    """
    preds_with_counts: predict_all_mda_with_counts_batched 의 출력(dict: ad_id -> df)
    tgt: TGT 윈도우의 raw logs (rpt_time_turn / rpt_time_clk 포함)
    반환: ads_idx, mda_idx, yhat_turn, score, tgt_turn 컬럼의 평가용 프레임
    """
    # 1) 예측 합치기
    pred_list=[]
    for ad, df in preds_with_counts.items():
        if df is None or df.empty: 
            continue
        tmp = df[["mda_idx","yhat_turn","score"]].copy()
        tmp["ads_idx"] = int(ad)
        pred_list.append(tmp)
    pred_df = pd.concat(pred_list, ignore_index=True) if pred_list else pd.DataFrame(
        columns=["ads_idx","mda_idx","yhat_turn","score"]
    )
    pred_df["ads_idx"] = pred_df["ads_idx"].astype(int)
    pred_df["mda_idx"] = pred_df["mda_idx"].astype(int)

    # 2) 실제(집계)
    t = tgt.copy()
    t["turn"] = pd.to_numeric(t.get("rpt_time_turn", 0), errors="coerce").fillna(0)
    gt = (t.groupby(["ads_idx","mda_idx"], as_index=False)
            .agg(tgt_turn=("turn","sum")))
    gt["ads_idx"] = gt["ads_idx"].astype(int)
    gt["mda_idx"] = gt["mda_idx"].astype(int)

    # 3) 예측 기준으로 left-join (예측이 없는 조합은 평가에서 제외)
    eval_df = pred_df.merge(gt, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_turn":0.0})
    return eval_df

def regression_metrics_overall(eval_df: pd.DataFrame) -> dict:
    """
    eval_df: _build_eval_frame 결과물
    """
    y_true = eval_df["tgt_turn"].values.astype(float)
    y_pred = eval_df["yhat_turn"].values.astype(float)

    mae  = float(mean_absolute_error(y_true, y_pred))
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))   # 버전 호환
    r2   = float(r2_score(y_true, y_pred))

    # MAPE (0 나누기 보호)
    eps = 1e-8
    denom = np.where(np.abs(y_true) < eps, np.nan, np.abs(y_true))
    mape = float(np.nanmean(np.abs((y_true - y_pred) / denom)) * 100.0)

    return {"MAE": mae, "RMSE": rmse, "R2": r2, "MAPE(%)": mape}

def print_metrics(m: dict, title: str = "Overall regression metrics"):
    print(f"\n=== {title} ===")
    print(f"MAE : {m['MAE']:.2f}")
    print(f"RMSE: {m['RMSE']:.2f}")
    print(f"R²  : {m['R2']:.3f}")
    print(f"MAPE: {m['MAPE(%)']:.2f}%")


In [43]:
# 예측 끝난 뒤:
# preds_with_counts, tgt_sel 이 이미 존재한다고 가정

eval_df = _build_eval_frame(preds_with_counts, tgt_sel)
metrics = regression_metrics_overall(eval_df)
print_metrics(metrics, title="XGBoost (전체 조합 기준)")



=== XGBoost (전체 조합 기준) ===
MAE : 3.75
RMSE: 183.03
R²  : 0.176
MAPE: 286.95%


In [16]:
def get_expected_vs_actual_for_ad(
    preds_with_counts: Dict[int, pd.DataFrame],
    tgt_df: pd.DataFrame,
    ad_id: int,
    sort_by: str = "yhat_turn",
    top: int = 10
) -> pd.DataFrame:
    """
    특정 광고(ad_id)에 대해 예측된 상위 매체와
    TGT 실제 전환수를 합쳐 비교 테이블을 반환.
    """
    # 1) 예측 테이블 가져오기
    pred_df = preds_with_counts.get(int(ad_id))
    if pred_df is None or pred_df.empty:
        return pd.DataFrame(columns=["mda_idx", "yhat_turn", "tgt_turn", "score"])

    # 2) TGT에서 실제 전환/클릭 집계 (안전한 형변환)
    t = tgt_df.copy()
    t["ads_idx"] = pd.to_numeric(t["ads_idx"], errors="coerce").astype("Int64")
    t["mda_idx"] = pd.to_numeric(t["mda_idx"], errors="coerce").astype("Int64")
    t["rpt_time_turn"] = pd.to_numeric(t.get("rpt_time_turn", 0), errors="coerce").fillna(0.0)
    t["rpt_time_clk"]  = pd.to_numeric(t.get("rpt_time_clk", 0),  errors="coerce").fillna(0.0)

    actual = (
        t[t["ads_idx"] == int(ad_id)]
        .groupby("mda_idx", as_index=False)
        .agg(tgt_turn=("rpt_time_turn", "sum"),
             tgt_clk =("rpt_time_clk",  "sum"))
    )
    actual["mda_idx"] = actual["mda_idx"].astype(int)

    # 3) 예측과 실제 머지
    view = pred_df.merge(actual, on="mda_idx", how="left")
    view[["tgt_turn","tgt_clk"]] = view[["tgt_turn","tgt_clk"]].fillna(0.0)

    # 4) 정렬 & 상위 N
    sort_by = sort_by if sort_by in ("yhat_turn", "score") else "yhat_turn"
    view = view.sort_values(sort_by, ascending=False).reset_index(drop=True)
    if top is not None:
        view = view.head(int(top))

    # 5) 깔끔한 출력 컬럼
    cols = ["mda_idx","yhat_turn","tgt_turn","score","tgt_clk"]
    cols = [c for c in cols if c in view.columns]
    return view[cols].copy()


In [ ]:
ad_id = 446418
top5_cmp = get_expected_vs_actual_for_ad(
    preds_with_counts, tgt_sel,  # 필요시 tgt_large → tgt_sel 교체
    ad_id=ad_id, sort_by="yhat_turn", top=10
)

print(f"=== Ad {ad_id} 추천 매체사 Top-10 (예상 전환수 vs 실제 전환수) ===")
print(top5_cmp[["mda_idx","yhat_turn","tgt_turn","score"]])
# 클릭까지 보고 싶으면 ↓
# print(top5_cmp[["mda_idx","yhat_turn","tgt_turn","tgt_clk","score"]])

=== Ad 446418 추천 매체사 Top-10 (예상 전환수 vs 실제 전환수) ===
   mda_idx   yhat_turn  tgt_turn     score
0      563  171.018875     196.0  5.269762
1      562    4.411408       8.0  0.781793
2      634    1.319105       0.0  0.448809
3      371    0.770419       0.0  2.432785
4      270    0.500069       0.0 -0.866350
5      817    0.060028       0.0 -1.484042
6      854    0.042022       0.0 -0.044255
7      492    0.023868       0.0 -2.214164
8      442    0.008801       0.0 -2.400579
9       54    0.006918       0.0 -2.872267


In [41]:

df = top5_cmp.copy()

y_true = df["tgt_turn"].values
y_pred = df["yhat_turn"].values

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))  # 버전 호환 안전
r2 = r2_score(y_true, y_pred)

# MAPE (0 나누기 방지)
mask = y_true > 0
mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

print(f"MAE : {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²  : {r2:.3f}")
print(f"MAPE: {mape:.2f}%")


MAE : 3.13
RMSE: 8.00
R²  : 0.981
MAPE: 28.80%


In [ ]:
# -------------------------------
# 9-추가) 신규 매체사만 필터링 헬퍼
# -------------------------------
def _build_excluded_pairs_for_ad(
    ad_id: int,
    ads_time: pd.DataFrame,
    running_pairs: Optional[pd.DataFrame] = None,
    date_from: Optional[str] = None,
    date_to: Optional[str] = None,
    exclude_click_ge1: bool = True,
    exclude_any_history: bool = True
) -> Set[int]:
    """
    특정 광고(ad_id)에 대해 '제외해야 할 mda_idx' 집합을 만든다.
      - 과거 로그에 등장한 모든 페어(ads_idx=ad_id)  [exclude_any_history]
      - 과거 클릭수가 1 이상인 페어                   [exclude_click_ge1]
      - 현재 진행중 페어(running_pairs)
    """
    t = ads_time.copy()
    # 날짜 필터
    if date_from or date_to:
        t["date"] = pd.to_datetime(t["rpt_time_date"], errors="coerce")
        if date_from: t = t[t["date"] >= pd.to_datetime(date_from)]
        if date_to:   t = t[t["date"] <= pd.to_datetime(date_to)]
    # ad 고정
    t = t[pd.to_numeric(t["ads_idx"], errors="coerce").fillna(-1).astype(int) == int(ad_id)].copy()
    t["mda_idx"] = pd.to_numeric(t["mda_idx"], errors="coerce").fillna(-1).astype(int)

    excluded = set()
    if exclude_any_history and not t.empty:
        excluded.update(t["mda_idx"].dropna().astype(int).tolist())
    if exclude_click_ge1 and not t.empty:
        clk = pd.to_numeric(t.get("rpt_time_clk", 0), errors="coerce").fillna(0)
        excluded.update(t.loc[clk >= 1, "mda_idx"].dropna().astype(int).tolist())

    if running_pairs is not None and not running_pairs.empty:
        rp = running_pairs.copy()
        rp["ads_idx"] = pd.to_numeric(rp["ads_idx"], errors="coerce").fillna(-1).astype(int)
        rp["mda_idx"] = pd.to_numeric(rp["mda_idx"], errors="coerce").fillna(-1).astype(int)
        excluded.update(rp.loc[rp["ads_idx"] == int(ad_id), "mda_idx"].tolist())

    return {int(m) for m in excluded if m != -1}

def get_new_media_for_ad(
    preds_with_counts: Dict[int, pd.DataFrame],
    ad_id: int,
    ads_time: pd.DataFrame,
    running_pairs: Optional[pd.DataFrame] = None,
    date_from: Optional[str] = None,
    date_to: Optional[str] = None,
    sort_by: str = "yhat_turn",
    top: int = 20,
    media_meta: Optional[pd.DataFrame] = None  # 예: media_portfolio[["mda_idx","mda_name"]]
) -> pd.DataFrame:
    """
    예측 결과에서 해당 광고의 '신규 매체사'만 반환.
      - 신규 정의: 이력 無, 클릭≥1 無, 현재 진행중 아님
    """
    # 1) 예측 상위 rows
    cand = get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by=sort_by, top=None)
    if cand.empty:
        return cand

    # 2) 제외할 mda 집합 구성
    excluded_mdas = _build_excluded_pairs_for_ad(
        ad_id=ad_id,
        ads_time=ads_time,
        running_pairs=running_pairs,
        date_from=date_from,
        date_to=date_to,
        exclude_click_ge1=True,
        exclude_any_history=True
    )

    # 3) 신규만 필터
    out = cand[~cand["mda_idx"].astype(int).isin(excluded_mdas)].copy()

    # 4) 이름 붙이기(옵션)
    if media_meta is not None:
        cols = [c for c in ["mda_idx","mda_name"] if c in media_meta.columns]
        if "mda_idx" in cols and len(cols) >= 1:
            out = out.merge(media_meta[cols].drop_duplicates("mda_idx"), on="mda_idx", how="left")

    # 5) top N
    if top is not None:
        out = out.head(int(top))
    return out


In [ ]:
# (I) 특정 광고 Top-K - '신규 매체사'만
ad_id = 349591
# 필요하면 과거 이력 기준 기간을 제한할 수도 있음 (예: 2024-01-01 ~ 2025-08-19)
# date_from, date_to = "2024-01-01", "2025-08-19"
date_from, date_to = None, None

top5_new = get_new_media_for_ad(
    preds_with_counts=preds_with_counts,
    ad_id=ad_id,
    ads_time=ads_time,                 # 전체 로그
    running_pairs=running_pairs,       # 현재 진행중 페어
    date_from=date_from,
    date_to=date_to,
    sort_by="yhat_turn",
    top=5,
    # 매체명 붙이고 싶으면 아래 전달 (없으면 None)
    media_meta=(media_portfolio[["mda_idx","mda_name"]] if "media_portfolio" in globals() and "mda_name" in media_portfolio.columns else None)
)
print(top5_new[["mda_idx","yhat_turn","score", *(["mda_name"] if "mda_name" in top5_new.columns else [])]])


   mda_idx  yhat_turn     score
5       18   2.894660  0.835572
3      371   2.207727  1.454288
6       14   1.915840  0.768716
4      482   1.312190  1.391240
8      540   0.905387 -0.154614


In [ ]:
# -*- coding: utf-8 -*-
# =========================================================
#  Ad × Media 추천 + 전환수 예측 (CatBoostRanker + CatBoostRegressor)
#  - 누수 방지: 모든 피처는 OBS에서만 생성, 라벨은 TGT
#  - 클릭: Tweedie 회귀, CVR: logit 회귀
#  - 예측: yhat_turn = yhat_click × yhat_cvr
#  - 계약종료형(banned_mda) 제거, 현재 진행중 페어(running_pairs)는 포함
# =========================================================
# 필요 패키지:
#   pip install pandas numpy pyarrow scikit-learn catboost
# =========================================================


warnings.filterwarnings("ignore")

# -------------------------------
# (공통 유틸/전처리/후처리) — XGB 버전과 동일
# -------------------------------
def _to_num(s): return pd.to_numeric(s, errors="coerce").fillna(0)
def _to_dt(s):  return pd.to_datetime(s, errors="coerce")
def _norm_cat_col(df: pd.DataFrame, col: str) -> pd.DataFrame:
    if col in df.columns:
        df = df.copy()
        df[col] = df[col].astype(str).str.strip().str.lower()
        df[col] = df[col].replace({"<na>":"__na__", "nan":"__na__", "none":"__na__", "nat":"__na__"})
    return df
def _norm_cat(s): return pd.Series(s, copy=False).astype("string").str.strip().str.lower()
def _pick_col(cols, cand):
    for c in cand:
        if c in cols: return c
    return None
def _ensure_col(df: pd.DataFrame, out_col: str, cands: List[str]) -> pd.DataFrame:
    col = _pick_col(df.columns, cands)
    df = df.copy()
    if col is None: df[out_col] = "__na__"
    else:           df[out_col] = _norm_cat(df[col])
    return df
def _dedup_running_pairs(running_pairs: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
    if running_pairs is None or running_pairs.empty: 
        return None
    rp = running_pairs.copy()
    rp["ads_idx"] = rp["ads_idx"].astype(int)
    rp["mda_idx"] = rp["mda_idx"].astype(int)
    rp = rp.drop_duplicates(["ads_idx","mda_idx"])
    return rp

def split_obs_tgt_by_dates(ads_time: pd.DataFrame,
    obs_start: str="2025-07-25", tgt_start: str="2025-08-20",
    obs_end: Optional[str]=None, tgt_end: Optional[str]=None, tgt_max_days: int=6):
    t = ads_time.copy()
    t["date"] = _to_dt(t["rpt_time_date"])
    if t["date"].isna().all():
        raise ValueError("ads_time['rpt_time_date']에 유효한 날짜가 없습니다.")
    obs_start = pd.to_datetime(obs_start).normalize()
    tgt_start = pd.to_datetime(tgt_start).normalize()
    obs_end = (pd.to_datetime(obs_end).normalize() if obs_end is not None else (tgt_start - pd.Timedelta(days=1)).normalize())
    last_date = t["date"].max().normalize()
    tgt_end = (pd.to_datetime(tgt_end).normalize() if tgt_end is not None else min(last_date, (tgt_start + pd.Timedelta(days=tgt_max_days)).normalize()))
    obs = t[(t["date"] >= obs_start) & (t["date"] <= obs_end)].copy()
    tgt = t[(t["date"] >= tgt_start) & (t["date"] <= tgt_end)].copy()
    return obs, tgt, obs_start, tgt_end, (tgt_start - pd.Timedelta(days=1)).normalize()

def _build_time_signals(obs: pd.DataFrame) -> pd.DataFrame:
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["weekday"] = _to_dt(t["rpt_time_date"]).dt.weekday
    wd = t.groupby(["mda_idx","weekday"], as_index=False).agg(clk=("clk","sum"))
    wd["is_weekend"] = wd["weekday"].isin([5,6]).astype(int)
    g = wd.groupby("mda_idx", as_index=False).agg(clk_total=("clk","sum"))
    w = wd.merge(g, on="mda_idx", how="left")
    day_ratio = (w.assign(ratio=w["clk"]/w["clk_total"].replace(0,np.nan))
                   .sort_values(["mda_idx","ratio"], ascending=[True,False])
                   .groupby("mda_idx").head(1)[["mda_idx","ratio"]]
                   .rename(columns={"ratio":"mda_day_ratio"}))
    weekend_ratio = (w.groupby(["mda_idx","is_weekend"], as_index=False)
                       .agg(clk=("clk","sum")).merge(g,on="mda_idx",how="left"))
    weekend_ratio["mda_weekend_ratio"] = (weekend_ratio["clk"] /
                                          weekend_ratio["clk_total"].replace(0,np.nan)).fillna(0)
    weekend_ratio = weekend_ratio[weekend_ratio["is_weekend"]==1][["mda_idx","mda_weekend_ratio"]]
    out = day_ratio.merge(weekend_ratio, on="mda_idx", how="outer").fillna(0.0)
    out["mda_idx"] = out["mda_idx"].astype(int)
    return out

def build_attr_context(obs: pd.DataFrame, ads_list_with_domain: pd.DataFrame):
    t = obs.copy()
    t["clk"]  = _to_num(t.get("rpt_time_clk", 0))
    t["conv"] = _to_num(t.get("rpt_time_turn", 0))
    mda_all = (t.groupby("mda_idx", as_index=False)
                 .agg(mda_clk_global=("clk","sum"), mda_conv_global=("conv","sum")))
    mda_all["mda_cvr_global"] = (mda_all["mda_conv_global"] /
                                 mda_all["mda_clk_global"].replace(0,np.nan)).fillna(0.0)
    mu, sd = mda_all["mda_conv_global"].mean(), mda_all["mda_conv_global"].std(ddof=0)
    mda_all["mda_pop_norm"] = (mda_all["mda_conv_global"] - mu) / (sd + 1e-9)

    al = ads_list_with_domain[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    al["ads_category"] = _norm_cat(al["ads_category"])
    t2 = t.merge(al, on="ads_idx", how="left")
    cat_mda = (t2.groupby(["mda_idx","ads_category"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    cat_mda["cat_cvr"] = (cat_mda["conv"]/cat_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot = cat_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    cat_mda = cat_mda.merge(mda_tot, on="mda_idx", how="left")
    cat_mda["cat_share"] = (cat_mda["conv"]/cat_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    t_dom = t.merge(
        ads_list_with_domain[["ads_idx","ads_domain"]].drop_duplicates("ads_idx"),
        on="ads_idx", how="left"
    )
    t_dom["ads_domain"] = _norm_cat(t_dom["ads_domain"])
    dom_mda = (t_dom.groupby(["mda_idx","ads_domain"], as_index=False)
                 .agg(clk=("clk","sum"), conv=("conv","sum")))
    dom_mda["dom_cvr"] = (dom_mda["conv"]/dom_mda["clk"].replace(0,np.nan)).fillna(0.0)
    mda_tot2 = dom_mda.groupby("mda_idx", as_index=False).agg(mda_conv_total=("conv","sum"))
    dom_mda = dom_mda.merge(mda_tot2, on="mda_idx", how="left")
    dom_mda["dom_share"] = (dom_mda["conv"]/dom_mda["mda_conv_total"].replace(0,np.nan)).fillna(0.0)

    time_sig = _build_time_signals(obs)

    adv_ctx = None
    if "adv_idx" in ads_list_with_domain.columns:
        t_adv = t.merge(ads_list_with_domain[["ads_idx","adv_idx"]].drop_duplicates("ads_idx"),
                        on="ads_idx", how="left")
        adv_all = (t_adv.groupby("adv_idx", as_index=False)
                        .agg(adv_clk_global=("clk","sum"),
                             adv_conv_global=("conv","sum")))
        adv_all["adv_cvr_global"] = (adv_all["adv_conv_global"] /
                                     adv_all["adv_clk_global"].replace(0,np.nan)).fillna(0.0)
        mu2, sd2 = adv_all["adv_conv_global"].mean(), adv_all["adv_conv_global"].std(ddof=0)
        adv_all["adv_pop_norm"] = (adv_all["adv_conv_global"] - mu2) / (sd2 + 1e-9)
        adv_ctx = adv_all

    return {"mda_all": mda_all, "cat_mda": cat_mda, "dom_mda": dom_mda,
            "time_sig": time_sig, "adv_ctx": adv_ctx}

def pick_train_ads_from_tgt(tgt: pd.DataFrame, min_pos_in_tgt: int = 1,
                            max_train_ads: Optional[int] = 20000, rng_seed: int = 42) -> np.ndarray:
    t = tgt.copy()
    t["turn"] = _to_num(t.get("rpt_time_turn", 0))
    pos = (t.groupby("ads_idx")["turn"].apply(lambda s: (s > 0).sum())
             .reset_index(name="pos_cnt"))
    ok = pos[pos["pos_cnt"] >= max(1,int(min_pos_in_tgt))]["ads_idx"].astype(int).values
    if max_train_ads is not None and len(ok) > max_train_ads:
        rng = np.random.RandomState(rng_seed)
        ok = rng.choice(ok, size=max_train_ads, replace=False)
    return np.sort(ok)

def create_candidates(ads_list: pd.DataFrame, mda_all: pd.DataFrame, cat_mda: pd.DataFrame,
                      topC=10, add_random=2, cat_topM=5,
                      min_active_clk=200, min_active_conv=20,
                      rng_seed=42, ad_ids_filter=None, max_per_ad=20):
    mda_active = mda_all[(mda_all["mda_clk_global"]>=min_active_clk) |
                         (mda_all["mda_conv_global"]>=min_active_conv)].copy()
    if mda_active.empty: mda_active = mda_all.copy()
    mda_ids_global = (mda_active.sort_values("mda_conv_global", ascending=False)["mda_idx"]
                      .astype(int).values)
    cat_top = (cat_mda.sort_values(["ads_category","cat_cvr"], ascending=[True,False])
                      .groupby("ads_category").head(cat_topM)[["ads_category","mda_idx"]])
    cat2top = {c: g["mda_idx"].astype(int).tolist() for c,g in cat_top.groupby("ads_category")}
    rng = np.random.RandomState(rng_seed)
    ad_ids = (np.array(ad_ids_filter, dtype=int) if (ad_ids_filter is not None and len(ad_ids_filter)>0)
              else ads_list["ads_idx"].dropna().astype(int).unique())
    ad_meta = ads_list[["ads_idx","ads_category"]].drop_duplicates("ads_idx").copy()
    ad_meta["ads_idx"] = ad_meta["ads_idx"].astype(int)
    ad_meta["ads_category"] = ad_meta["ads_category"].astype(str).str.strip().str.lower()
    ad2cat = ad_meta.set_index("ads_idx")["ads_category"].to_dict()

    pairs=[]
    for ad in ad_ids:
        base=[]
        cat = ad2cat.get(int(ad), None)
        if cat in cat2top: base.extend(cat2top[cat])
        base.extend(mda_ids_global[:min(topC,len(mda_ids_global))])
        base=list(dict.fromkeys(base))
        remain=np.setdiff1d(mda_ids_global, np.array(base, dtype=int))
        if len(remain)>0 and add_random>0:
            base.extend(rng.choice(remain, size=min(add_random,len(remain)), replace=False).tolist())
        if len(base)>max_per_ad: base=base[:max_per_ad]
        pairs += [(int(ad), int(m)) for m in base]
    out = pd.DataFrame(pairs, columns=["ads_idx","mda_idx"]).drop_duplicates()
    print(f"[Candidates] pairs={len(out):,} | ads={len(ad_ids):,} | active_mda={len(mda_active):,} | topC={topC}, rnd={add_random}, cat_topM={cat_topM}, per_ad≤{max_per_ad}")
    return out

FEAT_ALLOW = [
    "mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm",
    "cat_cvr","cat_share",
    "price_reward_log","price_contract_log","price_ratio",
    "dom_cvr","dom_share",
    "mda_day_ratio","mda_weekend_ratio",
    "adv_cvr_global","adv_pop_norm",
    "mda_size","active_ratio","day_ratio","weekend_ratio"
]

def prepare_training_attr_only(
    obs: pd.DataFrame, tgt: pd.DataFrame, ads_list_with_domain: pd.DataFrame,
    topC=10, add_random=2, cat_topM=5, min_active_clk=200, min_active_conv=20,
    max_train_ads: Optional[int] = 20000, min_pos_in_tgt: int = 1,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None,
    running_pairs: Optional[pd.DataFrame] = None
):
    running_pairs = _dedup_running_pairs(running_pairs)
    if excluded_mda_ids:
        obs = obs[~obs["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()
        tgt = tgt[~tgt["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()
    train_ads = pick_train_ads_from_tgt(tgt, min_pos_in_tgt=min_pos_in_tgt, max_train_ads=max_train_ads)
    actx = build_attr_context(obs, ads_list_with_domain)
    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]

    cand = create_candidates(
        ads_list=ads_list_with_domain, mda_all=mda_all, cat_mda=cat_mda,
        topC=topC, add_random=add_random, cat_topM=cat_topM,
        min_active_clk=min_active_clk, min_active_conv=min_active_conv,
        ad_ids_filter=train_ads, max_per_ad=20
    )
    # 진행중 페어 포함
    if excluded_mda_ids:
        cand = cand[~cand["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    df = cand.merge(
        mda_all[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]],
        on="mda_idx", how="left"
    )
    ad_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    ad_meta = _norm_cat_col(ad_meta, "ads_category")
    ad_meta = _ensure_col(ad_meta, "ads_domain", ["ads_domain","domain"])
    for c in ["ads_reward_price","ads_contract_price"]:
        ad_meta[c] = _to_num(ad_meta[c])
    ad_meta["price_reward_log"]   = np.log1p(ad_meta["ads_reward_price"])
    ad_meta["price_contract_log"] = np.log1p(ad_meta["ads_contract_price"])
    ad_meta["price_ratio"]        = (ad_meta["ads_reward_price"] /
                                     ad_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)

    df = df.merge(ad_meta[["ads_idx","ads_category","ads_domain","adv_idx",
                           "price_reward_log","price_contract_log","price_ratio"]],
                  on="ads_idx", how="left")

    df      = _norm_cat_col(df, "ads_category")
    cat_mda = _norm_cat_col(cat_mda, "ads_category")
    df = df.merge(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]],
                  on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0,"cat_share":0.0})

    dom_mda = dom_mda.copy()
    dom_mda["ads_domain"] = _norm_cat(dom_mda["ads_domain"])
    df["ads_domain"] = _norm_cat(df["ads_domain"])
    df = df.merge(dom_mda[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                  on=["mda_idx","ads_domain"], how="left").fillna({"dom_cvr":0.0,"dom_share":0.0})

    time_sig = actx["time_sig"]
    df = df.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if actx["adv_ctx"] is not None:
        adv_ctx = actx["adv_ctx"].copy()
        df = df.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in df.columns: df[c]=0.0
    df[["adv_cvr_global","adv_pop_norm"]] = df[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: df = df.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in df.columns: df[c]=0.0

    tgt2 = tgt.copy()
    tgt2["clk"]  = _to_num(tgt2.get("rpt_time_clk", 0))
    tgt2["turn"] = _to_num(tgt2.get("rpt_time_turn",0))
    y_agg = (tgt2.groupby(["ads_idx","mda_idx"], as_index=False)
                   .agg(tgt_clk=("clk","sum"), tgt_turn=("turn","sum")))
    y_agg["tgt_cvr"] = (y_agg["tgt_turn"] / y_agg["tgt_clk"].replace(0,np.nan)).clip(0,1)
    df = df.merge(y_agg, on=["ads_idx","mda_idx"], how="left").fillna({"tgt_clk":0.0,"tgt_turn":0.0})

    y = df["tgt_turn"].astype(float)
    rel = np.zeros(len(df), dtype=int)
    pos = y > 0
    if pos.sum()>0:
        ylog = np.log1p(y[pos])
        try:
            bins = pd.qcut(ylog, q=3, labels=False, duplicates="drop")
            rel[pos] = bins.astype(int)+1
        except ValueError:
            rel[pos] = 1
    df["relevance"] = rel
    return df, actx

LABEL_GAIN = [0.0,1.0,4.0,12.0]

def ensure_feature_matrix(df, feat_cols):
    X = df.copy()
    for c in feat_cols:
        if c not in X.columns: X[c]=0.0
        if not np.issubdtype(X[c].dtype, np.number):
            X[c]=pd.to_numeric(X[c], errors="coerce")
    return X[feat_cols].fillna(0.0).astype(np.float32)

def downsample_negatives_per_ad(df, ratio=4, random_state=42):
    out, rng = [], np.random.RandomState(random_state)
    for ad, g in df.groupby("ads_idx", sort=False):
        pos = g[g["tgt_turn"]>0]; neg = g[g["tgt_turn"]<=0]
        if len(pos)==0: continue
        cap = len(pos)*ratio
        if len(neg)>cap: neg = neg.sample(n=cap, random_state=rng)
        out.append(pd.concat([pos,neg], ignore_index=True))
    return pd.concat(out, ignore_index=True) if out else df

# -------------------------------
# 6) 학습기(CatBoost)
# -------------------------------
def train_attr_ranker_cat(train_df: pd.DataFrame):
    feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
    X = ensure_feature_matrix(train_df, feat_cols).values
    y = train_df["relevance"].astype(int).values
    group_id = train_df["ads_idx"].astype(int).values
    pool = Pool(X, y, group_id=group_id, feature_names=feat_cols)

    ranker = CatBoostRanker(
        loss_function="YetiRank",      # 또는 QueryRMSE
        eval_metric="NDCG:top=10",
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=1.5,
        iterations=1200,
        random_seed=42,
        verbose=False
    )
    ranker.fit(pool)
    return ranker, feat_cols

def train_click_regressor_cat(train_df: pd.DataFrame, feat_cols: List[str], tweedie_p: float = 1.1):
    X = ensure_feature_matrix(train_df, feat_cols).values
    y = train_df["tgt_clk"].astype(float).values
    reg = CatBoostRegressor(
        loss_function=f"Tweedie:variance_power={tweedie_p}",
        eval_metric="RMSE",
        learning_rate=0.05, depth=6, l2_leaf_reg=1.5,
        iterations=1200, random_seed=42, verbose=False
    )
    reg.fit(X, y, feature_names=feat_cols)
    return reg, {"mode":"tweedie_click","twp":tweedie_p}

def _logit_clip(p, eps=1e-6):
    p = np.clip(p, eps, 1-eps)
    return np.log(p/(1-p))
def _sigmoid(x): return 1/(1+np.exp(-x))

def train_cvr_regressor_cat(train_df: pd.DataFrame, feat_cols: List[str]):
    g = train_df[train_df["tgt_clk"]>0].copy()
    if g.empty:
        dummy = CatBoostRegressor(iterations=10, verbose=False).fit(np.zeros((1, len(feat_cols))), [0.0])
        return dummy, {"mode":"cvr_logit","fallback":True, "mean_logit":-20.0}
    X = ensure_feature_matrix(g, feat_cols).values
    y = (g["tgt_turn"]/g["tgt_clk"]).clip(0,1).values
    y_logit = _logit_clip(y)

    reg = CatBoostRegressor(
        loss_function="RMSE",
        learning_rate=0.05, depth=6, l2_leaf_reg=1.5,
        iterations=1200, random_seed=42, verbose=False
    )
    reg.fit(X, y_logit, feature_names=feat_cols)
    return reg, {"mode":"cvr_logit","fallback":False}

# -------------------------------
# 7) 예측/평가/편의 — XGB 버전과 동일
# -------------------------------
def predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_with_domain,
    K=20, batch_size=5000, max_predict_ads: Optional[int] = 20000,
    media_portfolio: Optional[pd.DataFrame] = None,
    excluded_mda_ids: Optional[set] = None,
    running_pairs: Optional[pd.DataFrame] = None
) -> Dict[int, pd.DataFrame]:
    running_pairs = _dedup_running_pairs(running_pairs)
    mda_all, cat_mda, dom_mda = actx["mda_all"], actx["cat_mda"], actx["dom_mda"]
    time_sig, adv_ctx = actx["time_sig"], actx["adv_ctx"]

    mda_active = mda_all[(mda_all["mda_clk_global"]>=200) | (mda_all["mda_conv_global"]>=20)]
    if mda_active.empty: mda_active = mda_all
    if excluded_mda_ids:
        mda_active = mda_active[~mda_active["mda_idx"].astype(int).isin(excluded_mda_ids)].copy()

    mda_active = mda_active[["mda_idx","mda_clk_global","mda_conv_global","mda_cvr_global","mda_pop_norm"]].copy()
    mda_active["mda_idx"]=mda_active["mda_idx"].astype(int)
    mda_active = mda_active.merge(time_sig, on="mda_idx", how="left").fillna({"mda_day_ratio":0.0,"mda_weekend_ratio":0.0})

    if media_portfolio is not None:
        mp = media_portfolio.copy()
        cols = [c for c in ["mda_idx","mda_size","active_ratio","day_ratio","weekend_ratio"] if c in mp.columns]
        if "mda_idx" in cols: mda_active = mda_active.merge(mp[cols], on="mda_idx", how="left")
    for c in ["mda_size","active_ratio","day_ratio","weekend_ratio"]:
        if c not in mda_active.columns: mda_active[c]=0.0

    ads_meta = ads_list_with_domain[["ads_idx","ads_category","ads_reward_price","ads_contract_price","ads_domain","adv_idx"]].drop_duplicates("ads_idx").copy()
    if max_predict_ads is not None: ads_meta = ads_meta.head(int(max_predict_ads))
    ads_meta = _norm_cat_col(ads_meta, "ads_category")
    ads_meta = _ensure_col(ads_meta, "ads_domain", ["ads_domain","domain"])
    ads_meta["ads_idx"] = ads_meta["ads_idx"].astype(int)
    ads_meta["ads_domain"] = _norm_cat(ads_meta["ads_domain"])
    ads_meta["ads_reward_price"]   = _to_num(ads_meta["ads_reward_price"])
    ads_meta["ads_contract_price"] = _to_num(ads_meta["ads_contract_price"])
    ads_meta["price_reward_log"]   = np.log1p(ads_meta["ads_reward_price"])
    ads_meta["price_contract_log"] = np.log1p(ads_meta["ads_contract_price"])
    ads_meta["price_ratio"]        = (ads_meta["ads_reward_price"] / ads_meta["ads_contract_price"].replace(0,np.nan)).fillna(0.0)
    if adv_ctx is not None:
        ads_meta = ads_meta.merge(adv_ctx[["adv_idx","adv_cvr_global","adv_pop_norm"]], on="adv_idx", how="left")
    for c in ["adv_cvr_global","adv_pop_norm"]:
        if c not in ads_meta.columns: ads_meta[c]=0.0
    ads_meta[["adv_cvr_global","adv_pop_norm"]] = ads_meta[["adv_cvr_global","adv_pop_norm"]].fillna(0.0)

    cat_mda_local = _norm_cat_col(cat_mda[["mda_idx","ads_category","cat_cvr","cat_share"]].copy(), "ads_category")
    dom_mda_local = dom_mda.copy(); dom_mda_local["ads_domain"] = _norm_cat(dom_mda_local["ads_domain"])

    out={}
    ad_ids = ads_meta["ads_idx"].tolist()
    for i in range(0, len(ad_ids), batch_size):
        ad_batch = ads_meta.iloc[i:i+batch_size]
        grid = ad_batch[["ads_idx","ads_category","ads_domain","price_reward_log","price_contract_log","price_ratio",
                         "adv_idx","adv_cvr_global","adv_pop_norm"]] \
               .merge(mda_active, how="cross")

        # 진행중 페어 포함
        grid = grid.merge(cat_mda_local, on=["mda_idx","ads_category"], how="left").fillna({"cat_cvr":0.0,"cat_share":0.0})
        grid = grid.merge(dom_mda_local[["mda_idx","ads_domain","dom_cvr","dom_share"]],
                          on=["mda_idx","ads_domain"], how="left").fillna({"dom_cvr":0.0,"dom_share":0.0})

        X = ensure_feature_matrix(grid, feat_cols).values
        # CatBoost는 Pool 예측도 가능하지만 여기선 numpy로 바로
        score = ranker.predict(Pool(X))

        yhat_clk = np.maximum(0.0, CatBoostRegressor(verbose=False).predict(X) if False else 0.0)  # placeholder
        # ↑ 위 한 줄은 타입 힌트 회피용. 실제론 아래 회귀 모델 사용:
        yhat_clk = np.maximum(0.0, click_reg.predict(X))
        yhat_cvr_logit = cvr_reg.predict(X)
        yhat_cvr = np.clip(_sigmoid(yhat_cvr_logit), 0.0, 1.0)
        yhat_turn = yhat_clk * yhat_cvr

        grid2 = grid[["ads_idx","mda_idx","ads_category","mda_cvr_global","mda_pop_norm","mda_clk_global","mda_conv_global"]].copy()
        grid2["score"] = score.astype(float)
        grid2["yhat_turn"] = yhat_turn.astype(float)

        for ad, g in grid2.groupby("ads_idx", sort=False):
            top = g.nlargest(K, "score")
            out[int(ad)] = top.sort_values("score", ascending=False).reset_index(drop=True)
        print(f"[Predict+Count] processed ads {i+len(ad_batch)}/{len(ad_ids)}")
    return out

def _precision_recall_hit_at_k(preds_dict, gt_dict, k=10):
    P,R,H=[],[],[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        pred = df["mda_idx"].astype(int).tolist()[:k]
        true = set(gt_dict[ad]); 
        if not true: continue
        inter = len(set(pred)&true)
        P.append(inter/max(1,len(pred))); R.append(inter/len(true)); H.append(1.0 if inter>0 else 0.0)
    return (float(np.mean(P)) if P else 0.0,
            float(np.mean(R)) if R else 0.0,
            float(np.mean(H)) if H else 0.0)

def _map_at_k(preds_dict, gt_dict, k=10):
    APs=[]
    for ad, df in preds_dict.items():
        if ad not in gt_dict: continue
        true=set(gt_dict[ad]); 
        if not true: continue
        ranked = df["mda_idx"].astype(int).tolist()[:k]
        hits=cum_prec=0
        for i,m in enumerate(ranked, start=1):
            if m in true:
                hits+=1; cum_prec += hits/i
        APs.append(cum_prec/min(len(true),k) if hits>0 else 0.0)
    return float(np.mean(APs)) if APs else 0.0

def _auc_flat(preds_dict, gt_pairs_dict, score_col="score"):
    y_true, y_score = [], []
    for ad, df in preds_dict.items():
        true_mdas = set(gt_pairs_dict.get(ad, []))
        for _, r in df.iterrows():
            y_true.append(1 if int(r["mda_idx"]) in true_mdas else 0)
            y_score.append(float(r[score_col]))
    if len(set(y_true))<2: return 0.5
    try: return float(roc_auc_score(y_true, y_score))
    except Exception: return 0.5

def evaluate_all(preds_with_counts: Dict[int, pd.DataFrame], tgt: pd.DataFrame, k=10):
    gt_pairs = tgt.groupby("ads_idx")["mda_idx"].apply(lambda s: list(pd.Series(s).dropna().astype(int).unique())).to_dict()
    prec, rec, hit = _precision_recall_hit_at_k(preds_with_counts, gt_pairs, k=k)
    mapk = _map_at_k(preds_with_counts, gt_pairs, k=k)
    auc_rank = _auc_flat(preds_with_counts, gt_pairs, score_col="score")
    auc_reg  = _auc_flat(preds_with_counts, gt_pairs, score_col="yhat_turn")
    return {f"precision@{k}":prec, f"recall@{k}":rec, f"map@{k}":mapk,
            f"hit_rate@{k}":hit, "auc_ranker":auc_rank, "auc_regressor":auc_reg}

def get_expected_conversions_for_ad(preds_with_counts: Dict[int, pd.DataFrame],
                                    ad_id: int, sort_by="yhat_turn", top=20) -> pd.DataFrame:
    df = preds_with_counts.get(int(ad_id))
    if df is None or df.empty:
        return pd.DataFrame(columns=["mda_idx","score","yhat_turn"])
    sort_by = "yhat_turn" if sort_by not in ("yhat_turn","score") else sort_by
    out = df.sort_values(sort_by, ascending=False).copy()
    return out.head(top) if top is not None else out

# =========================================================
# 메인 플로우는 XGB 버전과 동일합니다(상단 주석의 호출 예시 참고).
# 학습기만 아래처럼 바꿔 호출하면 됩니다:
#
# train_df_ds = downsample_negatives_per_ad(train_df, ratio=4)
# ranker, feat_cols   = train_attr_ranker_cat(train_df_ds)
# click_reg, click_md = train_click_regressor_cat(train_df_ds, feat_cols, tweedie_p=1.1)
# cvr_reg, cvr_md     = train_cvr_regressor_cat(train_df_ds, feat_cols)
# preds_with_counts   = predict_all_mda_with_counts_batched(...)
# metrics_k5/10       = evaluate_all(...)
# =========================================================


In [ ]:


def train_attr_ranker_cat(train_df: pd.DataFrame):
    feat_cols = [c for c in FEAT_ALLOW if c in train_df.columns]
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["relevance"].astype(int).values
    group_id = train_df["ads_idx"].astype(int).values  # 각 행의 그룹(광고) ID

    train_pool = Pool(
        data=X,
        label=y,
        group_id=group_id,
        feature_names=feat_cols
    )

    ranker = CatBoostRanker(
        loss_function="YetiRank",
        iterations=1200,
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=1.5,
        random_seed=42,
        verbose=False
        # GPU 쓰면: task_type="GPU", devices="0"
    )
    ranker.fit(train_pool)
    return ranker, feat_cols


def train_click_regressor_cat(train_df: pd.DataFrame, feat_cols: List[str], tweedie_p: float = 1.1):
    X = ensure_feature_matrix(train_df, feat_cols)
    y = train_df["tgt_clk"].astype(float).values

    train_pool = Pool(
        data=X,
        label=y,
        feature_names=feat_cols
    )

    reg = CatBoostRegressor(
        loss_function=f"Tweedie:variance_power={tweedie_p}",
        eval_metric="RMSE",
        iterations=1200,
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=1.5,
        random_seed=42,
        verbose=False
        # GPU: task_type="GPU", devices="0"
    )
    reg.fit(train_pool)
    return reg, {"mode":"tweedie_click","twp":tweedie_p}


def train_cvr_regressor_cat(train_df: pd.DataFrame, feat_cols: List[str]):
    g = train_df[train_df["tgt_clk"] > 0].copy()
    if g.empty:
        # 데이터가 전부 0클릭이면 더미 모델 반환
        dummy = CatBoostRegressor(iterations=1, verbose=False)
        X_dummy = np.zeros((1, len(feat_cols)), dtype=np.float32)
        pool_dummy = Pool(data=X_dummy, label=[0.0], feature_names=feat_cols)
        dummy.fit(pool_dummy)
        return dummy, {"mode":"cvr_logit","fallback":True, "mean_logit":-20.0}

    X = ensure_feature_matrix(g, feat_cols)
    y = (g["tgt_turn"] / g["tgt_clk"].replace(0, np.nan)).clip(0, 1).fillna(0.0).values
    # CVR은 로짓 회귀
    eps = 1e-6
    y = np.clip(y, eps, 1 - eps)
    y_logit = np.log(y / (1 - y))

    train_pool = Pool(
        data=X,
        label=y_logit,
        feature_names=feat_cols
    )

    reg = CatBoostRegressor(
        loss_function="RMSE",      # 로짓 회귀는 RMSE로 충분
        iterations=1200,
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=1.5,
        random_seed=42,
        verbose=False
        # GPU: task_type="GPU", devices="0"
    )
    reg.fit(train_pool)
    return reg, {"mode":"cvr_logit","fallback":False}


In [ ]:
# (A) 윈도우
obs, tgt, start, end, cut = split_obs_tgt_by_dates(ads_time, obs_start="2025-07-25", tgt_start="2025-08-20")
print(f"[Window] OBS: {start.date()} → {cut.date()}  |  TGT: {(cut+pd.Timedelta(days=1)).date()} → {end.date()}")

# (A-1) 계약종료형
if 'media_performance_classification' in globals():
    banned_mda = set(
        media_performance_classification
        .loc[media_performance_classification["basic_classification"].astype(str).str.strip() == "계약종료형", "mda_idx"]
        .dropna().astype(int).unique()
    )
else:
    banned_mda = set()
print(f"[Exclude] 계약종료형 mda 개수 = {len(banned_mda)}")

# (A-2) OBS/TGT에서 계약종료형 제거
if banned_mda:
    obs = obs[~obs["mda_idx"].astype(int).isin(banned_mda)].copy()
    tgt = tgt[~tgt["mda_idx"].astype(int).isin(banned_mda)].copy()

# (A-3) 현재 진행중 페어는 포함 (⇒ 함수 인자엔 None을 넘긴다)
running_pairs = _dedup_running_pairs(running_pairs) if 'running_pairs' in globals() else None
print(f"[Include] running_pairs (현재 진행중 페어) = {0 if running_pairs is None else len(running_pairs)}")

# (B) ads_list 확장
ads_list_ext = ads_list.merge(
    ads_segment[["ads_idx","ads_size","domain"]].rename(columns={"domain":"ads_domain"}),
    on="ads_idx", how="left"
)

# (C) 사이즈 필터
size_whitelist = {"MEGA","LARGE","MEDIUM"}
ads_list_sel = ads_list_ext[
    ads_list_ext["ads_size"].astype(str).str.strip().str.upper().isin(size_whitelist)
].copy()
sel_ids = set(ads_list_sel["ads_idx"].astype(int))
tgt_sel  = tgt[tgt["ads_idx"].astype(int).isin(sel_ids)].copy()
print(f"[Filter] sizes kept = {sorted(size_whitelist)} | ads={ads_list_sel['ads_idx'].nunique():,} | tgt pairs={len(tgt_sel):,}")

# (D) 학습 데이터 (진행중 페어 포함하려면 running_pairs=None)
train_df, actx = prepare_training_attr_only(
    obs, tgt_sel, ads_list_sel,
    topC=10, add_random=2, cat_topM=5,
    min_active_clk=200, min_active_conv=20,
    max_train_ads=20000, min_pos_in_tgt=1,
    media_portfolio=None,
    excluded_mda_ids=banned_mda,
    running_pairs=None     # ★ 포함: None으로 넘김
)
print("train_df size:", len(train_df))

# (E) CatBoost 학습기들
train_df_ds = downsample_negatives_per_ad(train_df, ratio=4)
ranker, feat_cols   = train_attr_ranker_cat(train_df_ds)
click_reg, click_md = train_click_regressor_cat(train_df_ds, feat_cols, tweedie_p=1.1)
cvr_reg, cvr_md     = train_cvr_regressor_cat(train_df_ds, feat_cols)
print("feat_cols used:", feat_cols)

# (F) 예측 & 평가 (진행중 페어 포함 ⇒ None)
preds_with_counts = predict_all_mda_with_counts_batched(
    ranker, click_reg, cvr_reg, feat_cols, actx, ads_list_sel,
    K=20, batch_size=5000, max_predict_ads=20000, media_portfolio=None,
    excluded_mda_ids=banned_mda, running_pairs=None   # ★ 포함: None
)
metrics_k5  = evaluate_all(preds_with_counts, tgt_sel, k=5)
metrics_k10 = evaluate_all(preds_with_counts, tgt_sel, k=10)
print("Eval@5:", metrics_k5)
print("Eval@10:", metrics_k10)

# (G) Sanity Check
leftover_tgt = tgt_sel["mda_idx"].astype(int).isin(banned_mda).sum()
print(f"[Sanity] TGT(selected sizes) 내 계약종료형 잔여: {leftover_tgt}")
any_banned_in_preds = any(
    int(m) in banned_mda
    for df in preds_with_counts.values()
    for m in df["mda_idx"].astype(int).tolist()
)
print(f"[Sanity] 예측결과에 계약종료형 포함 여부: {any_banned_in_preds}")


[Window] OBS: 2025-07-25 → 2025-08-19  |  TGT: 2025-08-20 → 2025-08-25
[Exclude] 계약종료형 mda 개수 = 51
[Include] running_pairs (현재 진행중 페어) = 0
[Filter] sizes kept = ['LARGE', 'MEDIUM', 'MEGA'] | ads=7,169 | tgt pairs=63,274
[Candidates] pairs=26,263 | ads=1,565 | active_mda=128 | topC=10, rnd=2, cat_topM=5, per_ad≤20
train_df size: 26263
feat_cols used: ['mda_clk_global', 'mda_conv_global', 'mda_cvr_global', 'mda_pop_norm', 'cat_cvr', 'cat_share', 'price_reward_log', 'price_contract_log', 'price_ratio', 'dom_cvr', 'dom_share', 'mda_day_ratio', 'mda_weekend_ratio', 'adv_cvr_global', 'adv_pop_norm', 'mda_size', 'active_ratio', 'day_ratio', 'weekend_ratio']
[Predict+Count] processed ads 5000/7169
[Predict+Count] processed ads 7169/7169
Eval@5: {'precision@5': 0.28511293634496926, 'recall@5': 0.7883134462067497, 'map@5': 0.7015337668263746, 'hit_rate@5': 0.898870636550308, 'auc_ranker': 0.8902008288689831, 'auc_regressor': 0.8247053899952035}
Eval@10: {'precision@10': 0.1677104722792608, 'reca

In [ ]:
# -------------------------------
# 9-추가) 신규 매체사만 필터링 헬퍼
# -------------------------------
def _build_excluded_pairs_for_ad(
    ad_id: int,
    ads_time: pd.DataFrame,
    running_pairs: Optional[pd.DataFrame] = None,
    date_from: Optional[str] = None,
    date_to: Optional[str] = None,
    exclude_click_ge1: bool = True,
    exclude_any_history: bool = True
) -> Set[int]:
    """
    특정 광고(ad_id)에 대해 '제외해야 할 mda_idx' 집합을 만든다.
      - 과거 로그에 등장한 모든 페어(ads_idx=ad_id)  [exclude_any_history]
      - 과거 클릭수가 1 이상인 페어                   [exclude_click_ge1]
      - 현재 진행중 페어(running_pairs)
    """
    t = ads_time.copy()
    # 날짜 필터
    if date_from or date_to:
        t["date"] = pd.to_datetime(t["rpt_time_date"], errors="coerce")
        if date_from: t = t[t["date"] >= pd.to_datetime(date_from)]
        if date_to:   t = t[t["date"] <= pd.to_datetime(date_to)]
    # ad 고정
    t = t[pd.to_numeric(t["ads_idx"], errors="coerce").fillna(-1).astype(int) == int(ad_id)].copy()
    t["mda_idx"] = pd.to_numeric(t["mda_idx"], errors="coerce").fillna(-1).astype(int)

    excluded = set()
    if exclude_any_history and not t.empty:
        excluded.update(t["mda_idx"].dropna().astype(int).tolist())
    if exclude_click_ge1 and not t.empty:
        clk = pd.to_numeric(t.get("rpt_time_clk", 0), errors="coerce").fillna(0)
        excluded.update(t.loc[clk >= 1, "mda_idx"].dropna().astype(int).tolist())

    if running_pairs is not None and not running_pairs.empty:
        rp = running_pairs.copy()
        rp["ads_idx"] = pd.to_numeric(rp["ads_idx"], errors="coerce").fillna(-1).astype(int)
        rp["mda_idx"] = pd.to_numeric(rp["mda_idx"], errors="coerce").fillna(-1).astype(int)
        excluded.update(rp.loc[rp["ads_idx"] == int(ad_id), "mda_idx"].tolist())

    return {int(m) for m in excluded if m != -1}

def get_new_media_for_ad(
    preds_with_counts: Dict[int, pd.DataFrame],
    ad_id: int,
    ads_time: pd.DataFrame,
    running_pairs: Optional[pd.DataFrame] = None,
    date_from: Optional[str] = None,
    date_to: Optional[str] = None,
    sort_by: str = "yhat_turn",
    top: int = 20,
    media_meta: Optional[pd.DataFrame] = None  # 예: media_portfolio[["mda_idx","mda_name"]]
) -> pd.DataFrame:
    """
    예측 결과에서 해당 광고의 '신규 매체사'만 반환.
      - 신규 정의: 이력 無, 클릭≥1 無, 현재 진행중 아님
    """
    # 1) 예측 상위 rows
    cand = get_expected_conversions_for_ad(preds_with_counts, ad_id, sort_by=sort_by, top=None)
    if cand.empty:
        return cand

    # 2) 제외할 mda 집합 구성
    excluded_mdas = _build_excluded_pairs_for_ad(
        ad_id=ad_id,
        ads_time=ads_time,
        running_pairs=running_pairs,
        date_from=date_from,
        date_to=date_to,
        exclude_click_ge1=True,
        exclude_any_history=True
    )

    # 3) 신규만 필터
    out = cand[~cand["mda_idx"].astype(int).isin(excluded_mdas)].copy()

    # 4) 이름 붙이기(옵션)
    if media_meta is not None:
        cols = [c for c in ["mda_idx","mda_name"] if c in media_meta.columns]
        if "mda_idx" in cols and len(cols) >= 1:
            out = out.merge(media_meta[cols].drop_duplicates("mda_idx"), on="mda_idx", how="left")

    # 5) top N
    if top is not None:
        out = out.head(int(top))
    return out


In [ ]:
# (I) 특정 광고 Top-K - '신규 매체사'만
ad_id = 34045
# 필요하면 과거 이력 기준 기간을 제한할 수도 있음 (예: 2024-01-01 ~ 2025-08-19)
# date_from, date_to = "2024-01-01", "2025-08-19"
date_from, date_to = None, None

top5_new = get_new_media_for_ad(
    preds_with_counts=preds_with_counts,
    ad_id=ad_id,
    ads_time=ads_time,                 # 전체 로그
    running_pairs=running_pairs,       # 현재 진행중 페어
    date_from=date_from,
    date_to=date_to,
    sort_by="yhat_turn",
    top=5,
    # 매체명 붙이고 싶으면 아래 전달 (없으면 None)
    media_meta=(media_portfolio[["mda_idx","mda_name"]] if "media_portfolio" in globals() and "mda_name" in media_portfolio.columns else None)
)
print(top5_new[["mda_idx","yhat_turn","score", *(["mda_name"] if "mda_name" in top5_new.columns else [])]])


    mda_idx  yhat_turn     score
1       756  83.905899  1.397102
2       757  77.573059  1.292437
0       678  65.227638  1.626016
11      796  26.459175  0.192208
7      1022  17.695169  0.547206
